In [ ]:
# Instalar librerías necesarias
!pip install pandas requests

# Importar librerías
import requests
import pandas as pd
from google.colab import files

# Definir los términos de búsqueda
search_terms = ['chromecast', 'google home', 'apple tv', 'amazon fire tv']

# Definir los campos que queremos extraer
fields = ['id', 'title', 'price', 'base_price', 'original_price', 'currency_id', 'initial_quantity',
          'available_quantity', 'sold_quantity', 'condition', 'category_id', 'reviews_quantity',
          'reviews_average', 'brand']


# Obtener resultados de búsqueda
items = []
for term in search_terms:
    response = requests.get(f'https://api.mercadolibre.com/sites/MLA/search?q={term}&limit=50')
    if response.status_code == 200:
        items.extend(response.json().get('results', []))

# Obtener información detallada de cada ítem
detailed_items = []
for item in items:
    item_id = item.get('id')
    response = requests.get(f'https://api.mercadolibre.com/items/{item_id}')
    if response.status_code == 200:
        item_details = response.json()
        # Extraer solo los campos deseados
        filtered_details = {field: item_details.get(field, None) for field in fields}
        # Extraer información adicional de 'reviews' si está disponible
        if 'reviews' in item_details:
            filtered_details['reviews_quantity'] = item_details['reviews']['rating']['quantity']
            filtered_details['reviews_average'] = item_details['reviews']['rating']['average']
        detailed_items.append(filtered_details)

# Guardar resultados en un archivo CSV
df = pd.DataFrame(detailed_items)
df.to_csv('get_data.csv', index=False)

print("¡Datos guardados exitosamente en 'get_data.csv'!")

# Descargar el archivo CSV
files.download('get_data.csv')